# Imports

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import matplotlib.pyplot as plt

# закомментить эти строки перед тем, как выгружать в html
# plt.style.use('ggplot')
# get_ipython().run_line_magic('matplotlib', 'inline')
# from matplotlib_inline.backend_inline import set_matplotlib_formats
# set_matplotlib_formats('svg')

from tqdm import tqdm

from fitter import Fitter, get_common_distributions, get_distributions
from sklearn.model_selection import train_test_split

import random
import math

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [ ]:
def drop_bug_st(df, max_score):

    proxy = df.groupby('session_id').sum()
    index_ch = proxy[proxy['max_score'] <= max_score].index
    
    df = df[df['session_id'].isin(index_ch)]
    
    return df

def plot_scores(df):
    all_scores = np.array(df[['session_id', 'score']].groupby('session_id').sum())
    plt.xlabel('Число решенных задач')
    plt.ylabel('Количество студентов с таким скором')
    plt.hist(all_scores)
    plt.show()
    
def plot_students(df):
    all_students = np.array(df[['session_id', 'school_id']].groupby('school_id').count())
    plt.xlabel('Число студентов в школе')
    plt.ylabel('Количество школ с таким числом студентов')
    plt.hist(all_students)
    plt.show()
    
    plt.xlabel('Число студентов в школе')
    plt.ylabel('Количество школ с таким числом студентов')
    plt.hist(all_students[all_students < 200])    
    plt.show()
    
def parewise_stat(n):
    return n * (n - 1) / 2

def metric_sim_wrong_ans_weighted_other(df, tasks):
    
    '''
    tasks - список номеров заданий
    '''
    
    X = df.copy()
    X = X[X['verdict'] == 'wrong']
    
    ans_dict = dict(zip(list(X['school_id'].unique()), np.zeros(len(list(X['school_id'].unique())))))
    
    for sch in ans_dict.keys():
        Y = X[X['school_id'] == sch]
        for i in range(len(tasks)):
            one_more_dict = Counter(Y[Y['task_no'] == tasks[i]]['users_answer'])
            stat = np.array(list(one_more_dict.values()))
            ans_dict[sch] += np.sum(parewise_stat(stat[stat > 1]))
            
                    
    return np.array(list(ans_dict.values())), list(ans_dict.keys())

def plot_scatter_stud_vs_metric(studs, res):
    plt.scatter(studs, res)
    plt.title('Распределение метрики на исходных данных')
    plt.xlabel('Количество школьников в школе')
    plt.ylabel('Значение метрики')
    plt.show()
    
def plot_log_metric(log_all_t_weighted):
    plt.hist(log_all_t_weighted)
    plt.title('Распределение метрики на исходных данных')
    plt.xlabel('Значение метрики')
    plt.ylabel('Количество школ с такой метрикой')
    plt.show()
    
def get_best_distribution(log_all_t_weighted):
    data_train, data_test = train_test_split(log_all_t_weighted, test_size=0.3, random_state=42)
    f = Fitter(data_train,
           distributions= ['gamma',
                          'beta',
                          'burr',
                          'gengamma'])
    f.fit()
    f.summary()
    plt.show()
    
    best_dist = f.get_best(method = 'sumsquare_error')
    dist_name = list(best_dist.keys())[0]
    a, b, loc, scale = f.fitted_param['beta']
    print('Q-Q Plot на тренировочной выборке')
    res_train = ss.probplot(data_train, sparams=(a, b, loc, scale), dist='beta', plot=plt, fit=True)
    plt.show()
    print('Q-Q Plot на тестовой выборке')
    res_test = ss.probplot(data_test, sparams=(a, b, loc, scale), dist='beta', plot=plt, fit=True)
    plt.show()
    
    print(f'p-value того, что train и test принадлежат одному распределению: {ss.ks_2samp(data_train, data_test).pvalue}')
    
    params = ss.beta.fit(data_train)
    return ss.beta(*params)

def get_dict_of_wr_ans_probs(df):
    X = df.copy()
    X = X[X['verdict']=='wrong']
    all_wrong_ans_dict = {}
    for i in range(1, 9):
        all_wrong_ans_dict[i] = dict(Counter(X[X['task_no'] == i]['users_answer']))
        
    for key in all_wrong_ans_dict.keys():
        s_wrong_i = sum(all_wrong_ans_dict[key].values())
        for key1 in all_wrong_ans_dict[key].keys():
            all_wrong_ans_dict[key][key1] = all_wrong_ans_dict[key][key1] / s_wrong_i
            
    return all_wrong_ans_dict

def get_wrong_ans(num_task, num_studs, distribution_dict):
    '''
    num_task - номер задания
    num_studs - количество студентов
    distribution_dict - словарь с распределением (см. предыдущую функцию)
    '''
    draw = np.random.choice(list(distribution_dict[num_task].keys()), num_studs, p=list(distribution_dict[num_task].values()), replace=True)
    return draw

def f(obs):
    if obs == 0:
        return 'wrong'
    else:
        return 'ok'
    
def get_task_id(num_studs, num_tasks):
    ans = [1] * num_studs
    for i in range(2, num_tasks + 1):
        ans.extend([i] * num_studs)
        
    return ans

def true_students(n, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans):
    
    '''
    если генерировать честных студентов из вероятностей каждого задания
    '''
    
    dim = n * num_of_tasks
    results = ss.bernoulli(array_emp_dist_of_correct).rvs(size=(n, num_of_tasks))
    score = results.flatten()
    all_answers = get_wrong_ans(1, n, dict_emp_dist_of_wrong_ans)
    for i in range(2, num_of_tasks+1):
        all_answers = np.concatenate((all_answers, get_wrong_ans(i, n, dict_emp_dist_of_wrong_ans)))
        
    school_ids = dim * [school_id]
    stud_id = np.array(range(0, dim)) % n + 1
    
    task_id = get_task_id(n, num_of_tasks)
    
    d = {
        'session_id': stud_id, 'task_no': task_id, 'score': score, 'school_id': school_ids, 'users_answer': all_answers, 
    }
            
    d = pd.DataFrame(data=d)
    d['verdict'] = d['score'].apply(f)
    return d

def cheat_students(max_studs_in_cluster, clusters, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans):
    
    '''
    генерация нечестных студентов из вероятностей
    '''
    
    X = true_students(1, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans)
    X['session_id'] = X['session_id'] + 900
    
    for i in range(clusters):
        stud_in_sample_cluster = random.randint(2, max_studs_in_cluster)
        Y = true_students(1, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans)
        Y = pd.concat([Y] * stud_in_sample_cluster)
        Y['session_id'] = np.array(range(0, stud_in_sample_cluster * num_of_tasks)) // num_of_tasks + 1 + 1000 + 100 * i
        X = pd.concat([X, Y])
        
    return X.reset_index(drop=True)

def generate_cheat_school(share_of_true, n, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans, max_studs_in_cluster=6):
    '''
    share_of_true - доля честных учеников
    n - среднее количество всех учеников 
    num_of_tasks - количество заданий
    school_id - id школы
    array_emp_dist_of_correct - распределение вероятностей правильного решения каждого задания
    dict_emp_dist_of_wrong_ans - распределение неправильных ответов
    max_studs_in_cluster - максимум школьников в кластере
    '''
    
    n_true = int(n * share_of_true)
    n_cheat = n - n_true
    
    test_df_true = true_students(n_true, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans)
    
    num_of_clusters = random.randint(1, n_cheat // 2)
    max_cheat_studs_per_cluster = n_cheat // num_of_clusters + 1
    
    test_df_cheat = cheat_students(max_cheat_studs_per_cluster, num_of_clusters, num_of_tasks, school_id, array_emp_dist_of_correct, dict_emp_dist_of_wrong_ans)
    
    return pd.concat([test_df_true, test_df_cheat]).reset_index(drop=True)

def get_true_power(n, sample):
    left_border = n > np.mean(n) - np.std(n)
    right_border = n < np.mean(n) + np.std(n)
    x = sample * right_border * left_border
    y = n * right_border * left_border
    return y[y != 0], x[x != 0]

def power_experiment(emp_p, all_wr_ans_probs, crytical_value, share_of_true_studs=0.7, number_of_students=50, tasks=8, rep=100):
    stat_cheat = np.zeros(rep)
    students_at_cheat = np.zeros(rep)

    for i in tqdm(range(rep)):
        test_cheat_df = generate_cheat_school(share_of_true_studs, number_of_students, tasks, 1, emp_p, all_wr_ans_probs)
        up = metric_sim_wrong_ans_weighted_other(test_cheat_df, np.array(range(1, tasks+1)))[0]
        down = (test_cheat_df.shape[0] / tasks) ** 2
        stat_cheat[i] =  np.log(up / down)
        students_at_cheat[i] = test_cheat_df.shape[0] / tasks
        
    ad_studs, ad_power = get_true_power(students_at_cheat, stat_cheat)

    print(f'Доля обнаруженных нечестных школ: {len(ad_power[ad_power > crytical_value]) / len(ad_power)}')
    plt.hist(ad_power)
    plt.axvline(x=crytical_value, c='r')
    plt.show()
    
def df_school_with_metric(school_ids, num_students, metric_raw):
    metric_not_logged = metric_raw / num_students ** 2
    d = {'schools': school_ids, 'metric_not_logged': metric_not_logged}
    df_return = pd.DataFrame(data=d)
    df_return = df_return[df_return['metric_not_logged'] > 0]
    df_return['metric_logged'] = np.log(df_return['metric_not_logged'])
    return df_return

def aa_experiment(emp_p, all_wr_ans_probs, crytical_value, share_of_true_studs=1, tasks=8, rep=100):
    
    stat_cheat = np.zeros(rep)
    students_at_cheat = np.zeros(rep)

    for i in tqdm(range(rep)):
        number_of_students = random.randint(10, 500)
        test_df = true_students(number_of_students, tasks, 1, emp_p, all_wr_ans_probs)
        up = metric_sim_wrong_ans_weighted_other(test_df, np.array(range(1, tasks+1)))[0]
        down = (number_of_students) ** 2
        stat_cheat[i] =  np.log(up / down)
        
    print(f'Доля отмеченных честных школ: {len(stat_cheat[stat_cheat > crytical_value]) / len(stat_cheat)}')
    plt.hist(stat_cheat)
    plt.axvline(x=crytical_value, c='r')
    plt.show()
    
    return np.quantile(stat_cheat, q=0.95)
def shuffle_school(df):
    df_test = df[['session_id', 'school_id']]
    df_test = df_test.drop_duplicates(subset=['session_id'])
    df_test['school_id'] = np.random.permutation(df_test['school_id'].values)
    X = df.copy()
    X = X.merge(df_test, on='session_id')
    X = X.drop(columns=['school_id_x'])
    X = X.rename(columns={'school_id_y':'school_id'})
    return X

def new_aa_experiment(df, crytical_value, tasks=8, rep=1, alpha=0.95):
    
    res = 0
    false_alarm = 0

    stat_random = np.array([])
    
    for i in tqdm(range(rep)):
    
        test_df = shuffle_school(df)
        up = metric_sim_wrong_ans_weighted_other(test_df, np.array(range(1, tasks+1)))[0]
        down = (np.array(test_df[['session_id', 'school_id']].groupby('school_id').count()).flatten()) ** 2
        dim = min(len(up), len(down))
        stat_cheat = np.log(up[:dim] / down[:dim])
        stat_cheat = stat_cheat[stat_cheat > -100]

        stat_random = np.append(stat_random, stat_cheat)

        res += np.quantile(stat_cheat, q=1-alpha)

        false_alarm += len(stat_cheat[stat_cheat > crytical_value])

    plt.hist(stat_cheat)
    plt.title('Распределение метрики на случайных школах')
    plt.axvline(x=crytical_value, color='r')
    plt.xlabel('Значение метрики')
    plt.show()
    
    return res / rep, false_alarm, stat_random

def make_cheat_school(test_df, schools, share_of_cheat):
    dfs = test_df[test_df['school_id'] == np.random.choice(schools)]
    students = dfs['session_id'].unique()
    n_cheat = int(dfs.shape[0] * share_of_cheat)
    
    ans = dfs.copy()
    
    for i in range(n_cheat):
        to_append = dfs[dfs['session_id'] == np.random.choice(students)]
        ans = pd.concat([ans, to_append], ignore_index=True)
        
    return ans

def new_power_experiment(df, crytical_value, share_of_cheat=0.3, tasks=8, rep=100): 
    schools = df['school_id'].unique()
    test_df = shuffle_school(df)
    
    res_stats = np.zeros(rep)
    for j in tqdm(range(rep)):
        try:
            sample = make_cheat_school(test_df, schools, share_of_cheat)
            up = metric_sim_wrong_ans_weighted_other(sample, np.array(range(1, tasks+1)))[0]
            down = (sample.shape[0] // 8) ** 2
            res_stats[j] = np.log(up / down)
        except:
            res_stats[j] = -200
            continue
        
        
    plt.hist(res_stats[res_stats > -100])
    plt.title('Распределение метрики на искусственных нечестных школах')
    plt.axvline(x=crytical_value, color='r')
    plt.xlabel('Значение метрики')
    plt.show()
    
    power = len(res_stats[res_stats > crytical_value]) / len(res_stats)
    print(f'Доля найденных нечестых школ (а-ка мощность): {power}')
    # print(f'Подставим бутсрап 95 квантиль по перемешанным школам')

    return res_stats
def get_1_type_error_vs_power(rand, cheat, distrib):
    quantiles = np.ones(5) - np.array([0.01, 0.02, 0.03, 0.04, 0.05])
    ans = {
        'заявленная ошибка первого рода': [], 'ошибка первого рода по распределению': [], 'мощность, используя квантили распределения': [], 'мощность, используя квантили случайных школ': []
    }
    for quantile in quantiles:
        ans['заявленная ошибка первого рода'].append(1 - quantile)

        cryt_val_dist = distrib.ppf(q=quantile)
        f_type_error = len(rand[rand > cryt_val_dist]) / len(rand)
        ans['ошибка первого рода по распределению'].append(round(f_type_error, 3))

        power_dist = len(cheat[cheat > cryt_val_dist]) / len(cheat)
        ans['мощность, используя квантили распределения'].append(round(power_dist, 3))

        cryt_val_random = np.quantile(rand, q=quantile)
        power_random = len(cheat[cheat > cryt_val_random]) / len(cheat)
        ans['мощность, используя квантили случайных школ'].append(round(power_random, 3))

    df = pd.DataFrame(ans)
    return df
def ultimate_pipeline(data, bootstrap_quantile, alpha = 0.01, rep_aa=10, rep_b=1000, share_of_cheat=0.3):
    # data = pd.read_csv(path_data)
    # schools = pd.read_csv(path_schools)
    # data = pd.merge(data, schools, how='left', on='session_id')
    
    max_score = int(round((np.mean(data.groupby('session_id').sum()['max_score'])), 0))
    number_of_schools = len(data['school_id'].unique())
        
    data = drop_bug_st(data, max_score)
    
    emp_p = np.array(data.groupby('task_no').sum()['score']) / (data.shape[0] / max_score)
    all_wr_ans_probs = get_dict_of_wr_ans_probs(data)
        
    # print('==========================')
    # print('EDA')
    # scores distribution
    # plot_scores(data)
    # students distribution
    # plot_students(data)
    all_students = np.array(data[['session_id', 'school_id']].groupby('school_id').count())
    # print(f'Количество школ, где людей меньше 10: {len(all_students[all_students < 10])}, или {round(len(all_students[all_students < 10]) / len(all_students) * 100, 2)}%')
    print('==========================')
    
    # подсчет метрики для исходного датафрейма
    print('Подсчет метрики для исходного датафрейма')
    
    res, schools_ans = metric_sim_wrong_ans_weighted_other(data, np.array(range(1, max_score + 1))) 
    studs = np.array(data[['session_id', 'school_id']].groupby('school_id').count()['session_id'])[:len(res)]
    metric = np.log(res / studs ** 2)
    
    df_of_metrics = df_school_with_metric(schools_ans, studs, res)
    
    # скаттер-плот метрики от количества школьников
    plot_scatter_stud_vs_metric(studs, metric)
    
    for_plot = res / studs ** 2
    all_t_weighted = for_plot[for_plot > 0]
    log_all_t_weighted = np.log(all_t_weighted[all_t_weighted > 0])
    
    plot_log_metric(log_all_t_weighted)
    
    best_distribution = get_best_distribution(log_all_t_weighted)
    
    crytical_value = best_distribution.ppf(q=1 - alpha)
    
    print(f'Уровень значимости: {alpha}, критическое значение по распределению: {round(crytical_value, 3)} (область правая)')
           
    print('==========================')
    print(f'АА-тестирование с перемешиванием')
    new_aa_95_quantile, false_alarm, all_random_schools_metric = new_aa_experiment(data, crytical_value, tasks=8, rep=rep_aa, alpha=alpha)
    print(f'{alpha} квантиль для случайных школ: {round(new_aa_95_quantile, 3)}')
    print(f'Ошибка первого рода при уровне значимости {alpha}: {false_alarm / (number_of_schools * rep_aa)}')

    if bootstrap_quantile == True:
        crytical_value = new_aa_95_quantile
        
    print('==========================')
    print('Тестирование мощности с перемешиванием')
    
    all_cheat_schools_metric = new_power_experiment(data, crytical_value, rep=rep_b, share_of_cheat=share_of_cheat)

    # табличка ошибка 1ого рода VS мощность
    table_precision_vs_power = get_1_type_error_vs_power(all_random_schools_metric, all_cheat_schools_metric, best_distribution)
    
    df_of_metrics['pvalue'] = best_distribution.sf(df_of_metrics['metric_logged'])
    return df_of_metrics[df_of_metrics['metric_logged'] > crytical_value].sort_values(by=['metric_logged'], ascending=False), table_precision_vs_power
def rename_cols(df):

    '''
    функция переименовывает колонки в датафрейме
    df - исходный датафрейм
    ->pd.DataFrame
    '''
    old = df.columns[2:]
    new = ['ans' + str(old[num]) for num in range(len(old))]
    d = dict(zip(old, new))
    return df.rename(columns=d)

def subst_correct(df, tasks=8):

    '''
    функция чистит данные 
    df - исходный датафрейм
    tasks - количество заданий 

    ->pd.DataFrame
    '''

    X = np.array(df.iloc[:, 2:2+tasks])
    Y = np.array(df.iloc[:, 2+tasks:])

    for i in range(len(X)):
        for j in range(tasks):
            if X[i, j] == 1:
                Y[i, j] = str(0)

    df.iloc[:, 2+tasks:] = Y

    return df

# subst_correct(data_test, tasks)

def transform_data(data, school, tasks=8):
    data_test = data[data['school_id'] == school]
    data_test_wide_score = pd.pivot(data_test, index=['session_id','school_id'], columns = 'task_no',values = 'score').reset_index()
    data_test_wide_ans = pd.pivot(data_test, index=['session_id','school_id'], columns = 'task_no',values = 'users_answer').reset_index()
    data_test_wide_ans = rename_cols(data_test_wide_ans)
    if tasks == 8:
        data_test = pd.concat([data_test_wide_score, data_test_wide_ans.drop(['session_id', 'school_id'], axis=1)], axis = 1)
    
    else:
        data_test_wide_ans = data_test_wide_ans.drop(['session_id', 'school_id'], axis=1)
        data_test_wide_ans = data_test_wide_ans.drop(tasks, axis=1)
        data_test = pd.concat([data_test_wide_score, data_test_wide_ans], axis = 1)
    
    return subst_correct(data_test, tasks)

def build_model(data_test, num_tasks=8):
    data_test_trans = data_test.iloc[:, 2 + num_tasks:]
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    data_test_trans = enc.fit_transform(data_test_trans)
    # print(data_test_trans.shape)
    link = linkage(data_test_trans, 'average', 'cosine')
    # dn = dendrogram(link, color_threshold = -1, orientation = "right")               
    # plt.show()
    
    dist = link[:, 2]
    dist_rev = dist[::]
    idxs = range(1, len(dist) + 1)
    dim = data_test.shape[0] // 10 * 9
    # plt.plot(idxs[-dim:], dist_rev[-dim:], marker='o')
    # plt.title('Расстояние между объединяемыми кластерами')
    # plt.xlabel('Шаг объединения')
    # plt.ylabel('Расстояние')

    return data_test_trans, link

def parewise_stat(n):
    return n * (n - 1) / 2

def metric_sim_wrong_ans_weighted_other_cluster(df, max_score = 8):

    tasks = np.array(range(1, max_score + 1))
    
    '''
    tasks - список номеров заданий
    '''
    
    X = df.copy()
    X = X[X['verdict'] == 'wrong']
    
    ans_dict = dict(zip(list(X['cluster'].unique()), np.zeros(len(list(X['cluster'].unique())))))
    
    for sch in ans_dict.keys():
        Y = X[X['cluster'] == sch]
        for i in range(len(tasks)):
            one_more_dict = Counter(Y[Y['task_no'] == tasks[i]]['users_answer'])
            stat = np.array(list(one_more_dict.values()))
            ans_dict[sch] += np.sum(parewise_stat(stat[stat > 1]))
            
    to_pd = {
        'cluster': list(ans_dict.keys()), 
        'cluster_metric': list(ans_dict.values())
    }      
    
    return pd.DataFrame(data=to_pd)
def cluster_analysis(data, school_num, cth):

    data_sample = data[data['school_id'] == school_num]
    data_test = transform_data(data_sample, school_num)
    data_test_trans, link = build_model(data_test)

    # cth = float(input('Введите порог для кластеризации'))

    # print(f'Выбранный порог для разбиения: {cth}')

    # dn = dendrogram(link, color_threshold = cth, orientation = "right")

    n_tasks = 8
    data_test['cluster'] = fcluster(link, cth, criterion='distance')
    data_test['total'] = data_test.iloc[:, 2:2 + n_tasks].sum(axis=1)
    clusters = data_test.groupby('cluster').count().sort_values(by='total', ascending=False)

    id_cluster = data_test[['session_id', 'cluster']]
    data_sample = data_sample.merge(id_cluster, how='left', on='session_id')
    metric_cluster = metric_sim_wrong_ans_weighted_other_cluster(data_sample)

    return data_test, metric_cluster.sort_values(by='cluster_metric', ascending=False)
def get_dict_of_wr_ans_probs(df):
    X = df.copy()
    X = X[X['verdict'].isin(['wrong', 'none'])]
    all_wrong_ans_dict = {}
    for i in range(1, 9):
        all_wrong_ans_dict[i] = dict(Counter(X[X['task_no'] == i]['users_answer']))
        
    for key in all_wrong_ans_dict.keys():
        s_wrong_i = sum(all_wrong_ans_dict[key].values())
        for key1 in all_wrong_ans_dict[key].keys():
            all_wrong_ans_dict[key][key1] = all_wrong_ans_dict[key][key1] / s_wrong_i

    return all_wrong_ans_dict

def get_dict_of_ok_ans_probs(data, task_num = 8):
    df = data.copy()    
    cor_ans_dict = df[df['verdict'] == 'ok'].groupby('task_no').count() / (df.shape[0] / task_num)
    cor_ans_dict = cor_ans_dict.reset_index()
    cor_ans_dict = dict(zip(list(cor_ans_dict['task_no'].values), list(cor_ans_dict['id'].values)))

    return cor_ans_dict

# LH для конкретного кластера

def get_cluster_lh(data, ids, dict_ok, dict_wrong, tasks=8):
    
    LH = 0
    for id in ids:
        df_id = data.copy()
        df_id = df_id[df_id['session_id'] == id]
        for i in (range(1, tasks+1)):

            verdict = df_id[df_id['task_no'] == i]['verdict'].values[0]
            answer = df_id[df_id['task_no'] == i]['users_answer'].values[0]
            # try:
            if verdict == 'ok':
                LH += np.log(dict_ok[i])

            else:
                LH += np.log(dict_wrong[i][answer] * (1 - dict_ok[i]))
            # except:
            #    LH += np.log(0.5)

    return LH

# LH для фиксированного кол-ва школьников 

def get_lh_bootstrap(n, cor_ans_dict, wr_ans_dict, tasks=8):
    LH = 0
    for i in range(tasks):
        pvals = [cor_ans_dict[i + 1]]
        pvals.extend(list(np.array(list(wr_ans_dict[i + 1].values())) * (1 - cor_ans_dict[i + 1])))
        dist = ss.rv_discrete(values=(pvals, pvals))
        LH += np.sum(np.log(dist.rvs(size=n)))

    return LH

def get_lh_distribution(n, cor_ans_dict, wr_ans_dict, tasks=8, rep=1000):
    all_LH = np.zeros(rep)
    for i in range(rep):
        all_LH[i] = get_lh_bootstrap(n, cor_ans_dict, wr_ans_dict)

    return all_LH

def get_randomness_proba(obs, dist):
    return (np.sum(dist < obs)) / len(dist)
def convert_to_answer(dict_data):

    ans = pd.DataFrame({'school_cluster_id': -1, 'session_id': [0], 'LH': 1, 'prob_LH': 2})

    for school in dict_data.keys():
        for cluster in dict_data[school].keys():
            df = pd.DataFrame(dict_data[school][cluster])
            ans = pd.concat([ans, df],  ignore_index=True)

    ans = ans.iloc[1:, :]

    return ans.sort_values(by=['prob_LH', 'school_cluster_id'])


def plot_LH_dist(df):
    X = df.copy()
    plt.hist(X.drop_duplicates(subset=['school_cluster_id'])['prob_LH'])
    plt.title('Распределение логправдоподобия по кластерам')
    plt.xlabel('LH кластера')
    plt.ylabel('Количество кластеров')
    plt.show()
def uber_ultimate_pipeline(data_path, schools_path, alpha = 0.01, rep_aa=10, rep_b=500, rep_lh=1000, cth=0.3, bootstrap_quantile=False, share_of_cheat=0.3):

    data = pd.read_csv(data_path)
    schools = pd.read_csv(schools_path)
    data = pd.merge(data, schools, how='left', on='session_id')

    # metric estimation and bad boys picking
    X = data.copy()
    res_pipeline, table_precision_vs_power = ultimate_pipeline(X, rep_aa=rep_aa, rep_b=rep_b, bootstrap_quantile=bootstrap_quantile, alpha=alpha, share_of_cheat=share_of_cheat)

    susp_schools = list(res_pipeline['schools'].values)

    # precompute probs

    cor_ans_dict = get_dict_of_ok_ans_probs(data)
    wr_ans_dict = get_dict_of_wr_ans_probs(data)

    LH_probs_dict = {}
    for i in tqdm(range(2, 6)):
        LH_probs_dict[str(i)] = get_lh_distribution(i, cor_ans_dict, wr_ans_dict, rep=rep_lh)

    # clusterization

    Y = data.copy()
    Y = Y[Y['school_id'].isin(susp_schools)]

    # ans = get_clusterization_and_probs(Y, susp_schools, cor_ans_dict, wr_ans_dict, LH_probs_dict, cth)
    ans_res = {}

    for k in tqdm(range(len(susp_schools))):
        Z = data.copy()

        susp_school = susp_schools[k]

        df_s, df_c = cluster_analysis(Z, susp_school, cth)
        df_c = df_c[df_c['cluster_metric'] > 3]

        susp_clusters = list(df_c['cluster'].values)

        proxy_dict = {}

        for j in (range(len(susp_clusters))):
            W = data.copy()
            sm_cluster = susp_clusters[j]
            
            sm_ids = list(df_s[df_s['cluster'] == sm_cluster]['session_id'].values)
            
            num_of_studs = len(sm_ids)

            my_index = str(susp_school) + '_' + str(sm_cluster)

            if num_of_studs <= 25:

                try:
                    sm_lh = get_cluster_lh(Z, sm_ids, cor_ans_dict, wr_ans_dict)
                    sm_ls_probs = get_randomness_proba(sm_lh, LH_probs_dict[str(num_of_studs)])
                    proxy_dict[sm_cluster] = {'school_cluster_id': my_index, 'session_id': sm_ids, 'LH':sm_lh, 'prob_LH':sm_ls_probs}
                except:
                    LH_probs_dict[str(num_of_studs)] = get_lh_distribution(num_of_studs, cor_ans_dict, wr_ans_dict, rep=rep_lh)
                    sm_lh = get_cluster_lh(Z, sm_ids, cor_ans_dict, wr_ans_dict)
                    sm_ls_probs = get_randomness_proba(sm_lh, LH_probs_dict[str(num_of_studs)])
                    proxy_dict[sm_cluster] = {'school_cluster_id': my_index, 'session_id': sm_ids, 'LH':sm_lh, 'prob_LH':sm_ls_probs}

        ans_res[susp_school] = proxy_dict

    cluster_df_ans = convert_to_answer(ans_res) 
    cluster_df_ans = cluster_df_ans[cluster_df_ans['prob_LH'] < 1]
    plot_LH_dist(cluster_df_ans)   

    return res_pipeline, cluster_df_ans, table_precision_vs_power

# Experiments

## Олимпиада номер 2090

In [ ]:
path = 'C:/Sirius/contests/'
data_path = path + '2090/' + 'results.csv'
schools_path = path + '2090/' + 'schools.csv'

cheat_df, cluster_df, table_precision_vs_power = uber_ultimate_pipeline(data_path, schools_path, rep_aa=10, rep_b=1000, rep_lh=1000, cth=0.3, alpha=0.05, bootstrap_quantile=False, share_of_cheat=0.3)
